## Spectral energy distributions of stellar mass black hole binary companions

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table

from wsdb import WSDB

%matplotlib inline

In [3]:
smbhb = Table.read("../data/smbh-binaries.fits")

In [5]:
smbhb[["ra", "dec"]]

ra,dec
deg,deg
float64,float64
269.10022599209594,-38.26669289511154
279.9364542149469,-17.366310727520602
281.7605090516464,-15.171493503661512
276.10652564741713,-14.29306953042355
262.72550156880436,-10.322016672099979
266.0817890957307,-38.57297971617643
274.5526206580929,-35.08807468130967
282.8113737965639,-17.25704766292388


In [ ]:
def get_sed_info(source):
    
    
    wsdb = WSDB()
    ps1 = wsdb.select("""
        SELECT * 
        FROM gaia_dr2_aux.gaia_source_ps1_xm
        WHERE source_id = {0}
        """.format(source["source_id"]))
    des = wsdb.select("""
        SELECT *
        FROM gaia_dr2_aux.gaia_source_desdr1_xm
        WHERE source_id = {0}
        """.format(source["source_id"]))
    twomass = wsdb.select("""
        SELECT *
        FROM gaia_dr2_aux.gaia_source_2mass_xm
        WHERE source_id = {0}
        """.format(source["source_id"]))
    sdss = wsdb.select("""
        SELECT *
        FROM gaia_dr2_aux.gaia_source_sdss_xm
        WHERE source_id = {0}
        """.format(source["source_id"]))
    
    galex = wsdb.select("""
        SELECT *
        FROM gaia_dr2_aux.gaia_source_galex_bcs_xm
        WHERE source_id = {0}""".format(source["source_id"]))
    
    allwise = wsdb.select("""
        WITH y as (
            SELECT *
            FROM gaia_dr2_aux.gaia_source_allwise_xm
            WHERE source_id = {0})
        select y.*, w1mpro, w2mpro, w3mpro, w4mpro from y, allwise.main as a where a.source_id=y.w_source_id
        """.format(source["source_id"]))
    
    # galex http://www.galex.caltech.edu/researcher/techdoc-ch1.html
    
    # 
    
    passbands = dict([
        ("sloan_u", 3543.0),
        ("sloan_g", 4770.0),
        ("sloan_r", 6231.0),
        ("sloan_i", 7625.0),
        ("sloan_z", 9134.0),
        ("allwise_w1", 34000.0),
        ("allwise_w2", 46000.0),
        ("allwise_w3",120000.0),
        ("allwise_w4",220000.0),
        ("ps1_g", 4810.0),
        ("ps1_r", 6170.0),
        ("ps1_i", 7520.0),
        ("ps1_z", 8660.0),
        ("ps1_y", 9620.0),
        ("twomass_j", 12350),
        ("twomass_h", 16620),
        ("twomass_k", 21590),
        ("des_g", 4725.0),
        ("des_r", 6375.0),
        ("des_i", 7725.0),
        ("des_z", 9200.0),
        ("des_y", 10000.0),
        ("galex_fuv", 1528.0),
        ("galex_nuv", 2271.0),
    ])
    
    
    colours = dict()
    if galex is not None:
        for v in "nf":
            colours["galex_{}uv_mag".format(v)] = (
                passbands["galex_{}uv_mag".format(v)],
                galex["galex_{}uv_mag".format(v)][0],
                galex["galex_{}uv_magerr".format(v)][0],
                galex["galex_{}uv_flux".format(v)][0],
                galex["galex_{}uv_fluxerr".format(v)][0]
            )

    if sdss is not None:
        for v in "ugriz":
            colours["sloan_{}".format(v)] = (
                passbands["sloan_{}".format(v)],
                sdss["psfMag_{}".format(v)][0],
                sdss["psgMagErr_{}".format(v)][0],
                sdss["cModelFlux_{}".format(v)][0],
                sdss["cModelFluxIvar_{}".format(v)][0]**-0.5,                
            )
    
    if twomass is not None:
        for v in "jhk":
            colours["twomass_{}".format(v)] = (
                passbands["twomass_{}".format(v)],
                twomass["{}_m".format(v)][0],
                twomass["{}_msigcom".format(v)][0],
                np.nan,
                np.nan
            )
    
    if ps1 is not None:
        for v in "grizy":
            colours["ps1_{}".format(v)] = (
                passbands["ps1_{}".format(v)],
                ps1["{}psfmag".format(v)][0],
                ps1["{}psfmagerr".format(v)][0]
            )
    
    if des is not None:
        #WAVG_MAG_PSF_{G,R,I,Z,Y}
        for v in "grizy":
            colours["des_{}".format(v)] = (
                passbands["des_{}".format(v)],
                des["WAVG_MAG_PSF_{}".format(v.upper())][0],
                des["WAVG_MAGERR_PSF_{}".format(v.upper())][0]
            )
    
    if allwise is not None:
        for v in (1,2,3,4):
            colours["allwise_w{:.0f}mpro".format(v)] = (
                passbands["allwise_w{:.0f}".format(v)],
                allwise["w{:.0f}mpro".format(v)][0],
                np.nan
            )
    
    return colours
    
    
    
colours = get_sed_info(smbhb[candidates][0])
print(colours)
    
    

In [ ]:
#3543, 4770, 6231, 7625, 9134


def plot_sed(wsdb_source, ax=None):
    
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    else:
        fig = ax.figure
    
    # get wavelength centroids
    # column name, wavelenvth
    
    # Sloan: http://skyserver.sdss.org/dr2/en/proj/advanced/color/sdssfilters.asp
    # Others: http://coolwiki.ipac.caltech.edu/index.php/Central_wavelengths_and_zero_points
        

    